In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SIRA_601/MyCode

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.data import shepp_logan_phantom
from skimage.transform import radon
import matplotlib.animation as animation
from skimage.draw import rectangle
from skimage.draw import polygon

# Functions in python?

In [ ]:
import matplotlib.pyplot as plt

def input_scores():
    """รับคะแนนจากผู้ใช้แล้วคืนเป็น list"""
    scores = []
    print("Enter scores for 3 subjects:")
    for i in range(3):
        score = float(input(f"Subject {i + 1}: "))
        scores.append(score)
    return scores

def calculate_average(scores):
    """คำนวณค่าเฉลี่ยจาก list ของคะแนน"""
    return sum(scores) / len(scores)

def classify_result(average):
    """ให้ผลการประเมินผ่าน/ไม่ผ่านตามค่าเฉลี่ย"""
    if average >= 60:
        return "Pass"
    else:
        return "Fail"

def display_result(average, status):
    """แสดงผลคะแนนเฉลี่ยและผลการประเมิน"""
    print(f"\nAverage Score: {average:.2f}")
    print(f"Result: {status}")

def plot_scores(scores, average):
    """สร้างกราฟแสดงคะแนนของแต่ละวิชาและเส้นค่าเฉลี่ย"""
    subjects = ['Subject 1', 'Subject 2', 'Subject 3']
    plt.figure(figsize=(8, 5))
    plt.bar(subjects, scores, color='skyblue', label='Score')
    plt.axhline(y=average, color='red', linestyle='--', label=f'Average = {average:.2f}')
    plt.ylim(0, 100)
    plt.title('Student Scores')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(axis='y', linestyle=':', alpha=0.7)
    plt.tight_layout()
    plt.show()

def main():
    scores = input_scores()
    avg = calculate_average(scores)
    result = classify_result(avg)
    display_result(avg, result)
    plot_scores(scores, avg)

# เรียกใช้งาน
main()


In [ ]:
import matplotlib.pyplot as plt

def get_scores():
    """คืน dictionary ของชื่อวิชาและคะแนน (mock data)"""
    return {
        "Math": 88,
        "Physics": 92,
        "Chemistry": 85,
        "Biology": 79,
        "English": 91
    }

def calculate_average(score_dict):
    """คืนค่าเฉลี่ยของคะแนนทั้งหมด"""
    total = sum(score_dict.values())
    return total / len(score_dict)

def find_top_subject(score_dict):
    """หาวิชาที่ได้คะแนนมากที่สุด"""
    max_score = max(score_dict.values())
    return [subject for subject, score in score_dict.items() if score == max_score]

def plot_subject_scores(score_dict, highlight_subjects):
    """สร้าง bar chart และเน้นวิชาที่ได้คะแนนสูงสุดด้วยสีพิเศษ"""
    subjects = list(score_dict.keys())
    scores = list(score_dict.values())

    colors = ['green' if subject in highlight_subjects else 'skyblue' for subject in subjects]

    plt.figure(figsize=(10, 5))
    plt.bar(subjects, scores, color=colors)
    plt.title("Subject Scores")
    plt.ylabel("Score")
    plt.ylim(0, 100)
    plt.axhline(60, color='red', linestyle='--', alpha=0.5, label='Passing Score')
    plt.legend()
    plt.grid(axis='y', linestyle=':', alpha=0.5)
    plt.tight_layout()
    plt.show()

def display_summary(avg, top_subjects):
    print(f"Average Score: {avg:.2f}")
    print(f"Top Subject(s): {', '.join(top_subjects)}")

def main():
    scores = get_scores()
    avg = calculate_average(scores)
    top = find_top_subject(scores)
    display_summary(avg, top)
    plot_subject_scores(scores, top)

# เรียกใช้งาน
main()


In [ ]:
import os,glob

glob.glob(os.getcwd()+"/*.png")

In [ ]:
from skimage.io import imread
from skimage.color import rgb2gray
from skimage import measure
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


from skimage.io import imread
from skimage.color import rgb2gray, rgba2rgb
from skimage.util import invert
from skimage import measure
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def analyze_hotspots_from_image(image_path: str, threshold: int = 160, threshold_area: int = 30,
                                 show_plot: bool = True, invert_image: bool = False, cmap: str = "gray"):
    """
    Analyze hot or cold spots in a medical image, optionally with inversion and custom colormap.

    Parameters:
        image_path (str): Path to the image file (jpg, png, tif).
        threshold (int): Intensity threshold for detecting regions.
        threshold_area (int): Minimum area (in pixels) to include in the output.
        show_plot (bool): Display original and binary mask images.
        invert_image (bool): If True, invert grayscale image before thresholding.
        cmap (str): Colormap for display (e.g., 'gray', 'hot', 'viridis').

    Returns:
        pd.DataFrame: Stats of regions passing threshold and area conditions.
    """
    # Load and convert to grayscale
    image = imread(image_path)
    if image.ndim == 3 and image.shape[2] == 4:
        image = rgba2rgb(image)
    if image.ndim == 3:
        image = rgb2gray(image)
    image_scaled = (image * 255).astype(np.uint8)

    # Optional inversion
    if invert_image:
        original = image_scaled
        image_scaled = invert(image_scaled)


    # Thresholding (intensity < threshold for cold spots, > threshold for hot spots)
    binary_mask = image_scaled < threshold
    labeled = measure.label(binary_mask)
    regions = measure.regionprops(labeled, intensity_image=image_scaled)

    # Collect regions meeting area requirement
    results = []
    for region in regions:
        if region.area > threshold_area:
            results.append({
                'Total Count': region.intensity_image[region.image].sum(),
                'Mean Count': region.mean_intensity,
                'Area (pixels)': region.area
            })

    # Show images
    if show_plot:
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.title("Original Image" + (" (Inverted)" if invert_image else ""))
        if invert_image:
            plt.imshow(original, cmap='gray')
        else:
            plt.imshow(image_scaled, cmap='gray')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title(f"Detected Spots (Threshold < {threshold}, Area > {threshold_area})")
        plt.imshow(image_scaled, cmap='gray')
        plt.imshow(binary_mask, cmap=cmap, alpha=0.7)
        plt.axis('off')

        plt.tight_layout()
        plt.show()

    return pd.DataFrame(results)

df = analyze_hotspots_from_image(
    "Thyroid.jpeg",
    threshold=140,
    threshold_area=200,
    invert_image=False,
    cmap="Reds"
)


In [ ]:
analyze_hotspots_from_image(
    "renal.jpeg",
    threshold=140,
    threshold_area=200,
    invert_image=False,
    cmap="Reds"
)

In [ ]:
analyze_hotspots_from_image(
    "renal_inverse.jpg",
    threshold=140,
    threshold_area=200,
    invert_image=True,
    cmap="Reds"
)

# Types of parameters in function

In [ ]:
# Positional arguments
def greet(name, age):
    print(f"Hello {name}, you are {age} years old.")

greet("Alice", 30)


In [ ]:
# Default parameters
def greet(name, age=18):
    print(f"Hello {name}, you are {age} years old.")

greet("Bob")          # Uses default age = 18
greet("Charlie", 25)  # Overrides default age


In [ ]:
# Keyword arguments
def greet(name, age):
    print(f"Hello {name}, you are {age} years old.")

greet(age=22, name="Diana")  # Order doesn't matter


In [ ]:
# introduce * (Asterisk) and ** (Double asterisk)
area = 5 * 4
print(area)

stars = '*' * 5
print(stars)

repeat = [1] * 3
print(repeat)

def add_all(*numbers): # (positional args → *args)
    return sum(numbers)

print(add_all(1, 2, 3))

a, *b, c = [1, 2, 3, 4, 5]
print(a, b, c)

def print_info(**kwargs): # (keyword args → **kwargs)
    for key, value in kwargs.items():
        print(key, "=", value)

print_info(name="Alice", age=30)

defaults = {'font': 'Arial', 'size': 12}
settings = {**defaults, 'size': 14}
print(settings)

In [ ]:
# non-Keyword variable arguments (*args)
def add_numbers(*args):
    total = sum(args)
    print(f"Total: {total}")

add_numbers(1, 2, 3)
add_numbers(10, 20)


In [ ]:
# Keyword variable arguments (*kwargs)
def print_info(**kwargs):
    for key, value in kwargs.items():
        print(f"{key}: {value}")

print_info(name="Emma", age=28, country="USA")


In [ ]:
def full_example(name, age=18, *args, **kwargs):
    print(f"Name: {name}")
    print(f"Age: {age}")
    print(f"Additional positional args: {args}")
    print(f"Additional keyword args: {kwargs}")

full_example("Zoe", 21, "student", "engineer", city="Bangkok", hobby="coding")


# Debugging

In [ ]:
def multiply(a, b=2):
    return a * b

print(multiply(5))
print(multiply(3, 4))


In [ ]:
def greet(name="Guest", age):
    print(f"Hello {name}, you are {age} years old.")


In [ ]:
def greet(age, name="Guest"):
    print(f"Hello {name}, you are {age} years old.")


In [ ]:
def mystery(x, y):
    if x > y:
        return x - y
    elif x == y:
        return "Equal"
    else:
        print("No return executed")

result = mystery(2, 5) # Hard-coded output
print("Result:", result) # ให้สังเกตุการใช้ print and return ว่าควรใช้อะไรเมื่อไร


In [ ]:
def analyze_numbers(*args):
    total = 0
    for num in args:
        if isinstance(num, int):
            if num % 2 == 0:
                total += num
            else:
                total -= num
    return total

result = analyze_numbers(10, 3, 4, '7', 5, 8)
print("Final result:", result)



In [ ]:
def func(a, b=10, *args, **kwargs):
    print("a:", a)
    print("b:", b)
    print("args:", args)
    print("kwargs:", kwargs)

func(1, *(2, 3, 4), d=5, b=20) # recall positional and keyword

In [ ]:
func(1, 2, 3, 4, d=5, b=20)

In [ ]:
func(1, *(2, 3, 4), d=5)

In [ ]:
def normalize(value, min_value=0, max_value=100):
    if max_value == min_value:
        return 0
    return (value - min_value) / (max_value - min_value)


In [ ]:
print(normalize(75, 0, 100))
print(normalize(50, 50, 50))


# Import developed functions by myself

In [ ]:
glob.glob(os.getcwd()+"/../*")

In [ ]:
glob.glob(os.getcwd()+'/u*/*')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/SIRA_601/MyCode/utils') # add python path


In [ ]:
import my_utility

# simulate image
image = np.random.randint(0, 255, size=(200, 200)).astype(np.uint8)

# call my_utility.py
hotspots = my_utility.detect_hotspots(image, threshold=200)
profile = my_utility.analyze_intensity_profile(image, axis=1)

print("Hotspots Detected:", hotspots[:2])
print("Profile Shape:", profile.shape)

In [ ]:
import my_utility as myutil
myutil.analyze_intensity_profile(image, axis=1).shape